<a href="https://colab.research.google.com/github/SergChupin/NLP/blob/lesson_07/nlp_hw_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Урок 7. Сверточные нейронные сети для анализа текста
Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек

  2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
  
  2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
!pip install stop_words
!pip install pymorphy2
!pip install navec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=35301775de7f1edb7df52ff653ea8b611070604ee7e4065fcac827ed2175d631
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 1.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=87409b89dd51d6de13781df555780fdfd743882840d8277a87df6a1f5b9366c0
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
# pd.set_option('display.float_format', '{:.2f}'.format)

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from spacy.lang.ru.stop_words import STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, get_file
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant

In [3]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 4.3 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [13]:
max_words = 2000
max_len = 100

In [6]:
data = pd.read_excel('/content/отзывы за лето.xls')
le = LabelEncoder()
data['Rating'] = le.fit_transform(data['Rating'])
data.sample(5)

,Rating,Content,Date
18953,4,"Быстро и качественно, спасибо.",2017-06-29
16202,0,Установка вообще не возможна после сброса к за...,2017-07-19
17459,4,Все гуд,2017-07-11
7654,4,Удобное приложение,2017-08-03
1992,4,Нет слова,2017-08-11


In [7]:
sw = set((get_stop_words("ru") + list(STOP_WORDS))) - {'не', 'ни', 'нет'}
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt, exclude=exclude, sw=sw, morpher=morpher):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [8]:
data['preprocessed_text'] = data['Content'].apply(preprocess_text)
data.sample(5)

,Rating,Content,Date,preprocessed_text
8596,4,Шикардос,2017-08-01,шикардос
9390,4,Очень удобный и быстрый сервис! Спасибо,2017-07-31,удобный быстрый сервис
12296,3,"До поломки смартфона ,пользовался, все устраив...",2017-07-27,поломка смартфон пользоваться устраивалосейчас...
16134,4,Удобно,2017-07-19,удобно
2669,4,Очень удобное и мобильное приложение !!!!,2017-08-10,удобный мобильный приложение


In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
train_corpus = " ".join(data["preprocessed_text"])
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
len(tokens_filtered)

102261

In [15]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'не',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'телефон',
 'отличный']

In [16]:
vocabulary = {v: k for k, v in enumerate(tokens_filtered_top, 1)}

In [17]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [19]:
data_train = np.asarray([text_to_sequence(text, max_len) for text in data['preprocessed_text']], dtype=np.int32)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data['Rating'], test_size=0.3)

In [25]:
batch_size = 1024
epochs = 10

In [22]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [23]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=optimizer,
              metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           82048     
                                                                 
 activation (Activation)     (None, 96, 128)           0         
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 20)                2580      
                                                                 
 activation_1 (Activation)   (None, 20)                0         
                                                        

In [26]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)

In [27]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 2s 247ms/step - loss: 0.6377 - accuracy: 0.7790


Test loss: 0.6377280950546265
Test accuracy: 0.7789609432220459


In [29]:
!pip install natasha
import natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 165 kB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 41 kB 103 kB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=1d2131e290f94e428b4e52d728f09fec7380dcb3a556e5b0e2c974c16bbb4ec8
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [34]:
from natasha import (
Segmenter,
MorphVocab,
NewsEmbedding,
NewsMorphTagger,
NewsSyntaxParser,
NewsNERTagger,
PER,
NamesExtractor,
Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [35]:
embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = emb.get(word)
    if embedding_vector is not None:
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [36]:
embedding_matrix.shape

(2000, 300)

In [37]:
embedding_layer = Embedding(
    max_words,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [38]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [39]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          validation_split=0.1)

In [41]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 4s 558ms/step - loss: 0.6755 - accuracy: 0.7625


Test loss: 0.6755297183990479
Test accuracy: 0.7625040411949158


Получилось даже чуть-чуть хуже